In [222]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import ray
from ray import tune
from sklearn.model_selection import train_test_split
import yaml
import argparse
import shutil
from collections import namedtuple
import os
import datetime

# Custom Libraries
from utils.model import Qkernel
from utils.data_generator import DataGenerator
from utils.agent import TrainModel
from utils.plotter import Plotter
from utils.helper import to_python_native, gen_experiment_name, set_seed
from utils.classifier_net import ClassifierNet

In [223]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"MPS is available. Using device: {device}")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available. Using device: {device}")
else:
    device = torch.device("cpu")
    print(f"Neither MPS nor CUDA is available. Using CPU: {device}")

MPS is available. Using device: mps


In [224]:
# Read Configs
with open('configs/comparision/checkerboard.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [225]:
data_generator = DataGenerator(     
                                dataset_name = 'corners', 
                                file_path = "data/corners.npy",
                                n_samples = 60, 
                                noise = 0.1, 
                                num_sectors = 3, 
                                points_per_sector = 10, 
                                grid_size = 4, 
                                sampling_radius = 0.05,
                                n_pca_features=None
                              )

training_data, training_labels, testing_data, testing_labels = data_generator.generate_dataset() #train_test_split(features, target, test_size=0.50, random_state=42)
training_data = torch.tensor(training_data.to_numpy(), dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(testing_data.to_numpy(), dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(training_labels.to_numpy(), dtype=torch.int)
testing_labels = torch.tensor(testing_labels.to_numpy(), dtype=torch.int)

In [226]:
kernel = Qkernel(   
                    device = config['qkernel']['device'], 
                    n_qubits = 5, 
                    trainable = True, 
                    input_scaling = True, 
                    data_reuploading = True, 
                    ansatz = 'he', 
                    ansatz_layers = 5   
                )
    
agent = TrainModel(
                    kernel=kernel,
                    training_data=training_data,
                    training_labels=training_labels,
                    testing_data=testing_data,
                    testing_labels=testing_labels,
                    optimizer= 'gd',
                    lr= 0.1,
                    mclr= 0.01,
                    cclr= 0.01,
                    epochs = 50,
                    train_method= 'ccka',
                    target_accuracy=0.95,
                    get_alignment_every=10,  
                    validate_every_epoch=None, 
                    base_path='.',
                    lambda_kao=0.01,
                    lambda_co=0.01,
                    clusters=20
                )

Epochs:  5


In [227]:
trained_kernel, main_centroids, class_centroids = agent.fit_kernel(training_data, training_labels)
#all_centroids = torch.cat(main_centroids + class_centroids, dim=0)


Started Training
------------------------------------------------------------------
Epoch: 0th, Alignment: 0
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 1th, Alignment: 0
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 2th, Alignment: 0
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 3th, Alignment: 0
------------------------------------------------------------------
------------------------------------------------------------------
Epoch: 4th, Alignment: 0
------------------------------------------------------------------


In [228]:
all_centroids = torch.cat(class_centroids, dim=0)
x_0 = all_centroids.repeat_interleave(training_data.shape[0],dim=0)
x_1 = training_data.repeat(all_centroids.shape[0], 1)

In [229]:
trained_kernel.eval()
_matrix = trained_kernel(x_0, x_1).to(torch.float32).reshape(all_centroids.shape[0],training_data.shape[0])

In [230]:
class_1_indices = [0, 1, 2, 3, 4]
class_2_indices = [5, 6, 7, 8, 9]

In [231]:
class_1_scores = _matrix.T[:, class_1_indices].mean(dim=1)
class_2_scores = _matrix.T[:, class_2_indices].mean(dim=1)
preds = torch.where(class_1_scores > class_2_scores, 1, -1)

In [232]:
len(class_2_scores)

100

In [233]:
preds

tensor([ 1, -1,  1,  1, -1, -1,  1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1,
        -1, -1,  1, -1,  1,  1, -1, -1, -1, -1, -1,  1, -1,  1,  1, -1,  1, -1,
         1, -1, -1, -1, -1,  1, -1,  1,  1,  1, -1,  1, -1,  1, -1, -1, -1, -1,
        -1, -1,  1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1,  1, -1, -1, -1, -1,
         1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1, -1,  1,  1, -1, -1, -1,
         1,  1, -1, -1,  1, -1, -1,  1, -1, -1])

In [234]:
accuracy = (preds == training_labels).float().mean()
print(f"Accuracy: {accuracy.item() * 100:.2f}%")

Accuracy: 49.00%


In [235]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')  # Now input is m-dimensional
clf.fit(_matrix.T.detach().numpy(), training_labels)

SVC()

In [236]:
X = _matrix.T.detach().float()         # [100, 12]
y = training_labels.long()   
dataset = TensorDataset(X, y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

x_0 = all_centroids.repeat_interleave(testing_data.shape[0],dim=0)
x_1 = testing_data.repeat(all_centroids.shape[0], 1)
_matrix_test = trained_kernel(x_0, x_1).to(torch.float32).reshape(all_centroids.shape[0],testing_data.shape[0])
X = _matrix.T.detach().float()         # [100, 12]
y = testing_labels.long()   
dataset = TensorDataset(X, y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [237]:
from sklearn.metrics import accuracy_score

predictions = clf.predict(_matrix_test.T.detach().numpy())
accuracy = accuracy_score(training_labels, predictions)
print(f"Training Accuracy: {accuracy * 100:.2f}%")

Training Accuracy: 48.00%
